# Analytical solution of temperature distribution in cylinder

In [1]:
from sympy import *
import IPython.display as disp

### Variable definition

In [2]:
r, r_e, l = symbols("r, r_e, ell")  # dimensions
sigma, lmbd, eps = symbols("sigma, lambda, epsilon")  # material properties
mu_0, sigma_sb = symbols("mu_0, sigma_sb")  # physical constants
t_ext, t_surf = symbols("T_ext, T_surf", real=True, positive=True)  # temperatures
H, omega = symbols("H, omega")  # electromagnetic properties

values = [
    (r_e, 0.06),
    (sigma, 58.8e3),
    (eps, 0.7),
    (sigma_sb, 5.6704e-8),
    (t_ext, 300),
    (H, 3*100/0.05),  # N*I/l
    (mu_0, 4*pi*1e-7),
    (omega, 2 * pi* 13.5e3),
    (lmbd, 65),
    (l, 0.05)
]

### Compute heat source

In [3]:
delta =  sqrt(2 / (mu_0 * sigma * omega ))
# disp.display(delta)
# delta = symbols("delta")
m = sqrt(2) *r_e / delta
xi = r / r_e
j = sqrt(-I)*sqrt(2) * H / delta * besselj(1, sqrt(-I) * m * xi) / besselj(0, sqrt(-I) * m)
disp.display(j)

h = 1/sigma * abs(j)**2 / 2  # heat source
# h = 1/sigma * j * conjugate(j) / 2  # heat source
disp.display(h.subs(r, r_e).subs(values).evalf(chop=True))  # this should be 1656752.9516706455 (as computed with scipy)

q = integrate(h*r*2*pi*l, (r, 0, r_e))  # total heat in W
disp.display(Eq(symbols("Q"), q))
disp.display(q.subs(values).evalf(chop=True))

# # compute power according to Lupi 2017 p.367 eqn 6.17
# P = re(-sqrt(-I) * besselj(1, sqrt(-I) * m) / besselj(0, sqrt(-I) * m))
# q_lupi = H**2 / 2 / sigma / delta * 2**0.5 * P * 2 * pi * r_e * l
# disp.display(q_lupi.subs(values).evalf())

H*sqrt(-I)*besselj(1, r*sqrt(-I)/sqrt(1/(mu_0*omega*sigma)))/(sqrt(1/(mu_0*omega*sigma))*besselj(0, r_e*sqrt(-I)/sqrt(1/(mu_0*omega*sigma))))

1656752.95167065 + 2.66453525910038e-15*I

Eq(Q, pi*ell*Integral(r*Abs(H*besselj(1, r*sqrt(-I)/sqrt(1/(mu_0*omega*sigma)))/besselj(0, r_e*sqrt(-I)/sqrt(1/(mu_0*omega*sigma))))**2, (r, 0, r_e))/(sigma*Abs(sqrt(1/(mu_0*omega*sigma)))**2))

273.760247555939

### Compute temperature of outer surface

In [4]:
# surface temperature

# eq_t_surf = Eq(q, eps*sigma_sb*(t_surf**4 - t_ext**4)*2*pi*r_e*l)
# # disp.display(eq_t_surf)
# solution = solveset(eq_t_surf, t_surf)
# # disp.display(simplify(solution))
# disp.display(solution.subs(values).evalf())

t_surf = root((q/(2*pi*r_e*l)+eps*sigma_sb*t_ext**4)/ (sigma_sb * eps), 4)
disp.display(t_surf)
disp.display(t_surf.subs(values).evalf(chop=True))

((T_ext**4*epsilon*sigma_sb + Integral(r*Abs(H*besselj(1, r*sqrt(-I)/sqrt(1/(mu_0*omega*sigma)))/besselj(0, r_e*sqrt(-I)/sqrt(1/(mu_0*omega*sigma))))**2, (r, 0, r_e))/(2*r_e*sigma*Abs(sqrt(1/(mu_0*omega*sigma)))**2))/(epsilon*sigma_sb))**(1/4)

782.017921881341

### Compute temperature distribution

In [5]:
# temperature distribution
t_out = symbols("T_out")

t = symbols("T", cls=Function)
h_r = symbols("h", cls=Function)
# diffeq = Eq(diff(t(r), r, r) + 1/r *diff(t(r), r), -h_r(r)/lmbd)
diffeq = Eq(diff(t(r), r, r) + 1/r *diff(t(r), r), -h/lmbd)
disp.display(diffeq)

disp.display(Eq(Derivative(t(0), r), 0))
disp.display(Eq(t(r_e), t_out))

# t_solution_raw = dsolve(diffeq, t(r))  # works
# disp.display(t_solution_raw) 
# disp.display(Derivative(t_solution_raw, r).doit())
# disp.display(Eq(0,diff(t_solution_raw, r).subs(r, 0)))

t_solution = dsolve(diffeq, t(r), ics={t(r_e): t_out})  # works
disp.display(t_solution)
eq_bc2 = Eq(0,diff(t_solution, r).subs(r, 0))
disp.display(eq_bc2)

# t_solution = dsolve(diffeq, t(r), ics={t(r).diff(r).subs(r, 0): 0})  # fails

# full solution
# t_solution = dsolve(diffeq, t(r), ics={t(r_e): t_out, t(r).diff(r).subs(r, 0): 0})
# disp.display(t_solution)

Eq(Derivative(T(r), (r, 2)) + Derivative(T(r), r)/r, -Abs(H*besselj(1, r*sqrt(-I)/sqrt(1/(mu_0*omega*sigma)))/besselj(0, r_e*sqrt(-I)/sqrt(1/(mu_0*omega*sigma))))**2/(2*lambda*sigma*Abs(sqrt(1/(mu_0*omega*sigma)))**2))

Eq(Derivative(T(0), r), 0)

Eq(T(r_e), T_out)

Eq(T(r), (2*lambda*sigma*(C2*log(r) - C2*log(r_e) + T_out + log(r_e)*Integral(r_e*Abs(H**2*mu_0*omega*sigma*besselj(1, r_e*sqrt(-I)/sqrt(1/(mu_0*omega*sigma)))**2/besselj(0, r_e*sqrt(-I)/sqrt(1/(mu_0*omega*sigma)))**2), r_e)/(2*lambda*sigma) - Integral(r_e*log(r_e)*Abs(H**2*mu_0*omega*sigma*besselj(1, r_e*sqrt(-I)/sqrt(1/(mu_0*omega*sigma)))**2/besselj(0, r_e*sqrt(-I)/sqrt(1/(mu_0*omega*sigma)))**2), r_e)/(2*lambda*sigma)) - log(r)*Integral(r*Abs(H**2*mu_0*omega*sigma*besselj(1, r*sqrt(-I)/sqrt(1/(mu_0*omega*sigma)))**2/besselj(0, r_e*sqrt(-I)/sqrt(1/(mu_0*omega*sigma)))**2), r) + Integral(r*log(r)*Abs(H**2*mu_0*omega*sigma*besselj(1, r*sqrt(-I)/sqrt(1/(mu_0*omega*sigma)))**2/besselj(0, r_e*sqrt(-I)/sqrt(1/(mu_0*omega*sigma)))**2), r))/(2*lambda*sigma))

Eq(0, Subs(Derivative(Eq(T(r), (2*lambda*sigma*(C2*log(r) - C2*log(r_e) + T_out + log(r_e)*Integral(r_e*Abs(H**2*mu_0*omega*sigma*besselj(1, r_e*sqrt(-I)/sqrt(1/(mu_0*omega*sigma)))**2/besselj(0, r_e*sqrt(-I)/sqrt(1/(mu_0*omega*sigma)))**2), r_e)/(2*lambda*sigma) - Integral(r_e*log(r_e)*Abs(H**2*mu_0*omega*sigma*besselj(1, r_e*sqrt(-I)/sqrt(1/(mu_0*omega*sigma)))**2/besselj(0, r_e*sqrt(-I)/sqrt(1/(mu_0*omega*sigma)))**2), r_e)/(2*lambda*sigma)) - log(r)*Integral(r*Abs(H**2*mu_0*omega*sigma*besselj(1, r*sqrt(-I)/sqrt(1/(mu_0*omega*sigma)))**2/besselj(0, r_e*sqrt(-I)/sqrt(1/(mu_0*omega*sigma)))**2), r) + Integral(r*log(r)*Abs(H**2*mu_0*omega*sigma*besselj(1, r*sqrt(-I)/sqrt(1/(mu_0*omega*sigma)))**2/besselj(0, r_e*sqrt(-I)/sqrt(1/(mu_0*omega*sigma)))**2), r))/(2*lambda*sigma)), r), r, 0))

In [12]:
c2 = symbols("C2")
disp.display(solveset(eq_bc2.subs(values), c2))

ValueError: 
Absolute values cannot be inverted in the complex domain.

In [8]:
# t_final = t_solution.subs(t_out, t_surf).subs(values)
# disp.display(t_final)
# disp.display(t_final.evalf())